<a href="https://colab.research.google.com/github/kavmut/Python-For-Finance-Portfolio-Optimization/blob/master/Python_For_Finance_Portfolio_Optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Description This program attemts to optimize a users portfolio using the Efficient Frontier

In [ ]:
# Import the libraries
from pandas_datareader import data as web
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

In [ ]:
# Get the stock symbols in the portfolio
# FAANG
assets = ['FB', 'AMZN', 'AAPL', 'NFLX', 'GOOG']

In [ ]:
# Assign weights to the stocks 
weights = np.array([0.2, 0.2, 0.2, 0.2, 0.2])

In [ ]:
# Get the stock / portfolio starting date
stockStartDate = '2013-01-01'

In [ ]:
# Get the stocks ending date (today)
today = datetime.today().strftime('%Y-%m-%d')
today

In [ ]:
# Create a dataframe to store the adjusted close price of the stocks
df = pd.DataFrame()

# Store the Adjusted close price of the stock indo the DataFrame
for stock in assets:
  df[stock] = web.DataReader(stock, data_source='yahoo', start=stockStartDate, end = today)['Adj Close']

In [ ]:
# Show the DataFrame
df

In [ ]:
# Visually show the stock / portfolio
title = 'Portfolio Adj. Close Price History'

# Get the stocks
my_stocks = df

# Create and plot the graph
for c in my_stocks.columns.values:
  plt.plot(my_stocks[c], label = c)

plt.title(title)
plt.xlabel('Date', fontsize = 18)
plt.ylabel('Adj. Price USD ($)', fontsize = 18)
plt.legend(my_stocks.columns.values, loc='upper left')
plt.show()

In [ ]:
# Sho the daily simple returns
returns = df.pct_change()
returns

In [ ]:
# Create and show the annualized covariance matrix
cov_matrix_annual = returns.cov() * 252
cov_matrix_annual

In [ ]:
# Calculate the portfolio variance
port_variance = np.dot( weights.T, np.dot(cov_matrix_annual, weights))
port_variance

In [ ]:
# Calcualte the portfolio volatility AKA standard deviation
port_volatility = np.sqrt(port_variance)
port_volatility

In [ ]:
# Calcualte the annual portfolio return
portflioSimpleAnnualReturn = np.sum(returns.mean() * weights) * 252
portflioSimpleAnnualReturn

In [ ]:
# Show the expected annual return, volatility, and variance

percent_var = str( round(port_variance, 2) * 100) + '%'
percent_vols = str(round(port_volatility,2)* 100 )+ '%'
percent_ret = str(round(portflioSimpleAnnualReturn, 2) * 100) + '%'

print('Expected annual return: '+ percent_ret)
print('Annual volatility / risk: '+ percent_vols)
print('Annual variance: '+ percent_var)

In [ ]:
pip install PyPortfolioOpt

In [ ]:
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns

In [ ]:
# Portfolio Optimization !

# Calculate the expected returns and the annualised sample covarinace matrix of asset returns
mu = expected_returns.mean_historical_return(df)
S = risk_models.sample_cov(df)

# optimize for max sharp ratio
ef = EfficientFrontier(mu, S)
weights = ef.max_sharpe()
cleaned_weights = ef.clean_weights()
print(cleaned_weights)
ef.portfolio_performance(verbose = True)

In [ ]:
# Get the discrete allocation of each share per stock
from pypfopt.discrete_allocation import DiscreteAllocation, get_latest_prices

latest_prices = get_latest_prices(df)
weights = cleaned_weights
da = DiscreteAllocation(weights, latest_prices, total_portfolio_value = 1500)

allocation , leftover = da.lp_portfolio()
print('Discrete allocation:', allocation)
print('Funds remaining: ${:.2f}'.format(leftover))